*Copyright (c) Microsoft Corporation. All rights reserved.*  
*Licensed under the MIT License.*

# Named Entity Recognition Using Transformer Model

## Summary

This notebook demonstrates how to fine tune [pretrained Transformer model](https://github.com/huggingface/transformers) for named entity recognition (NER) task. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, and model evaluation. 

The pretrained transformer of [BERT (Bidirectional Transformers for Language Understanding)](https://arxiv.org/pdf/1810.04805.pdf) architecture is used in this notebook. [BERT](https://arxiv.org/pdf/1810.04805.pdf) is a powerful pre-trained lanaguage model that can be used for multiple NLP tasks, including text classification, question answering, named entity recognition, etc. It's able to achieve state of the art performance with only a few epochs of fine tuning on task specific datasets.

The figure below illustrates how BERT can be fine tuned for NER tasks. The input data is a list of tokens representing a sentence. In the training data, each token has an entity label. After fine tuning, the model predicts an entity label for each token in a given testing sentence. 

<img src="https://nlpbp.blob.core.windows.net/images/bert_architecture.png">

In [4]:
# Ensure edits to libraries are loaded and plotting is shown in the notebook.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

/anaconda/envs/nlp_gpu/lib/python3.6/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
import os
import random
import string
import sys
from tempfile import TemporaryDirectory

import pandas as pd
import scrapbook as sb
import torch
from seqeval.metrics import classification_report
from sklearn.model_selection import train_test_split
from utils_nlp.common.pytorch_utils import dataloader_from_dataset
from utils_nlp.common.timer import Timer
from utils_nlp.dataset import wikigold
from utils_nlp.dataset.ner_utils import read_conll_file
from utils_nlp.dataset.url_utils import maybe_download
from utils_nlp.models.transformers.named_entity_recognition import (
    TokenClassificationProcessor, TokenClassifier)
from utils_nlp.models.transformers.named_entity_recognition import supported_models as SUPPORTED_MODELS


## Configuration

The running time shown in this notebook is on a Standard_NC12 Azure Virtual Machine with 2 NVIDIA Tesla K80 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

The table below provides some reference running time on different machine configurations.  

|QUICK_RUN|Machine Configurations|Running time|
|:---------|:----------------------|:------------|
|True|4 CPUs, 14GB memory| ~ 2 minutes|
|False|4 CPUs, 14GB memory| ~1.5 hours|
|True|1 NVIDIA Tesla K80 GPUs, 12GB GPU memory| ~ 1 minute|
|False|1 NVIDIA Tesla K80 GPUs, 12GB GPU memory| ~ 7 minutes |

If you run into CUDA out-of-memory error or the jupyter kernel dies constantly, try reducing the `BATCH_SIZE` and `MAX_SEQ_LENGTH`, but note that model performance will be compromised. 

In [6]:
# Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = False

In [7]:
# fraction of the dataset used for testing
TEST_DATA_FRACTION = 0.3

# sub-sampling ratio
SAMPLE_RATIO = 1

# the data path used to save the downloaded data file
DATA_PATH = TemporaryDirectory().name

# the cache data path during find tuning
CACHE_DIR = TemporaryDirectory().name

# set random seeds
RANDOM_SEED = 100
torch.manual_seed(RANDOM_SEED)

# model configurations
NUM_TRAIN_EPOCHS = 5
MODEL_NAME = "distilbert-base-cased"
DO_LOWER_CASE = False
MAX_SEQ_LENGTH = 200
TRAILING_PIECE_TAG = "X"
NUM_GPUS = None  # uses all if available
BATCH_SIZE = 16

# update variables for quick run option
if QUICK_RUN:
    SAMPLE_RATIO = 0.1
    NUM_TRAIN_EPOCHS = 1

### Models that can be used for token classification task

In [8]:
pd.DataFrame({"supported models": SUPPORTED_MODELS})

,supported models
0,albert-base-v1
1,albert-base-v2
2,albert-large-v1
3,albert-large-v2
4,albert-xlarge-v1
...,...
65,xlm-roberta-large-finetuned-conll02-spanish
66,xlm-roberta-large-finetuned-conll03-english
67,xlm-roberta-large-finetuned-conll03-german
68,xlnet-base-cased


## Get Training & Testing Dataset

The dataset used in this notebook is the [wikigold dataset](https://www.aclweb.org/anthology/W09-3302). The wikigold dataset consists of 145 mannually labelled Wikipedia articles, including 1841 sentences and 40k tokens in total. The dataset can be directly downloaded from [here](https://github.com/juand-r/entity-recognition-datasets/tree/master/data/wikigold). 

In the following cell, we download the data file, parse the tokens and labels, sample a given number of sentences, and split the dataset for training and testing.

In [39]:
train_df, test_df = wikigold.load_train_test_dfs(test_fraction=TEST_DATA_FRACTION)
train_df.head(10)

Maximum sequence length is: 144


,sentence,labels
0,"[He, is, currently, a, Fellow, of, All, Souls,...","[O, O, O, O, O, O, I-ORG, I-ORG, I-ORG, O, I-O..."
1,"[Coronado, Unified, School, District, is, the,...","[I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, I-..."
2,[-DOCSTART-],[O]
3,"[Original, Hinkley, locomotives, #, 1-2, were,...","[O, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O,..."
4,"[[, Surfwear, label, ], Quiksilver, totally, p...","[O, O, O, O, I-ORG, O, O, O, O, O, O, O, O, O,..."
5,"[The, Boilerhouse, Boys, is, the, stage, name,...","[O, I-ORG, I-ORG, O, O, O, O, O, O, O, I-PER, ..."
6,"[Success, would, follow, him, at, his, every, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,"[Norton, connected, first, and, very, hard, .]","[I-PER, O, O, O, O, O, O]"
8,"[Budjana, also, owns, a, double, neck, Klein, ...","[I-PER, O, O, O, O, O, I-ORG, O, O, O, O, O, O..."
9,"["", Leaders, must, be, close, enough, to, rela...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [40]:
pd.DataFrame({"text": train_df.sentence[0], "labels": train_df.labels[0]})

,text,labels
0,He,O
1,is,O
2,currently,O
3,a,O
4,Fellow,O
5,of,O
6,All,I-ORG
7,Souls,I-ORG
8,College,I-ORG
9,at,O


The following is an example input sentence of the training set.

> If your data is unlabeled, try using an annotation tool to simplify the process of labeling. The example [here](../annotation/Doccano.md) introduces [Doccanno](https://github.com/chakki-works/doccano) and shows how it can be used for NER annotation.

## Create PyTorch Datasets and Dataloaders
Given the tokenized input and corresponding labels, we use a custom processer to convert our input lists into a PyTorch dataset that can be used with our token classifier. Next, we create PyTorch dataloaders for training and testing.

In [47]:
train_dataloader, test_dataloader, label_map, test_dataset = wikigold.load_dataset(
    local_path=DATA_PATH, 
    test_fraction=TEST_DATA_FRACTION, 
    random_seed=RANDOM_SEED,
    model_name=MODEL_NAME,
    to_lower = DO_LOWER_CASE,
    max_len=MAX_SEQ_LENGTH,
    batch_size=BATCH_SIZE,
    num_gpus=NUM_GPUS)

Maximum sequence length is: 144


## Train Model

There are two steps to train a NER model using pretrained transformer model: 1) Instantiate a TokenClassifier class which is a wrapper of a transformer-based network, and 2) Fit the model using the preprocessed training dataloader. The member method `fit` of TokenClassifier class is used to fine-tune the model.

In [53]:
# Instantiate a TokenClassifier class for NER using pretrained transformer model
model = TokenClassifier(
    model_name=MODEL_NAME,
    num_labels=len(label_map),
    cache_dir=CACHE_DIR
)

# Fine tune the model using the training dataset
with Timer() as t:
    model.fit(
        train_dataloader=train_dataloader,
        num_epochs=NUM_TRAIN_EPOCHS,
        num_gpus=NUM_GPUS,
        local_rank=-1,
        weight_decay=0.0,
        learning_rate=5e-5,
        adam_epsilon=1e-8,
        warmup_steps=0,
        verbose=False,
        seed=RANDOM_SEED
    )

print("Training time : {:.3f} hrs".format(t.interval / 3600))



Training time : 0.058 hrs


## Evaluate on Testing Dataset

The `predict` method of the TokenClassifier returns a Numpy ndarray of raw predictions. The shape of the ndarray is \[`number_of_examples`, `sequence_length`, `number_of_labels`\]. Each value in the ndarray is not normalized. Post-process will be needed to get the probability for each class label. Function `get_predicted_token_labels` will process the raw prediction and output the predicted labels for each token.

In [54]:
with Timer() as t:
    preds = model.predict(
        test_dataloader=test_dataloader,
        num_gpus=None,
        verbose=True
    )

print("Prediction time : {:.3f} hrs".format(t.interval / 3600))

Scoring: 100%|██████████| 35/35 [00:06<00:00,  5.72it/s]

Prediction time : 0.002 hrs


Get the true token labels of the testing dataset:

In [55]:
true_labels = model.get_true_test_labels(label_map=label_map, dataset=test_dataset)

Get the predicted labels for each token by calling member method `get_predicted_token_labels`, and generate the classification report.

In [56]:
predicted_labels = model.get_predicted_token_labels(
    predictions=preds,
    label_map=label_map,
    dataset=test_dataset
)

report = classification_report(true_labels, 
              predicted_labels, 
              digits=2
)

print(report)

           precision    recall  f1-score   support

     MISC       0.65      0.74      0.69       186
      LOC       0.84      0.85      0.85       316
      ORG       0.76      0.82      0.79       270
      PER       0.92      0.92      0.92       261

micro avg       0.77      0.84      0.81      1033
macro avg       0.80      0.84      0.82      1033



## Score Example Sentences
Finally, we test the model on some random input sentences.

In [57]:
# test
sample_text = [    
    "Is it true that Jane works at Microsoft?",
    "Joe now lives in Copenhagen."
]
sample_tokens = [x.split() for x in sample_text]

processor = TokenClassificationProcessor()
sample_dataset = processor.preprocess(
    text=sample_tokens,
    max_len=MAX_SEQ_LENGTH,
    labels=None,
    label_map=label_map,
    trailing_piece_tag=TRAILING_PIECE_TAG,
)
sample_dataloader = dataloader_from_dataset(
    sample_dataset, batch_size=BATCH_SIZE, num_gpus=None, shuffle=False, distributed=False
)
preds = model.predict(
        test_dataloader=sample_dataloader,
        num_gpus=None,
        verbose=True
)
predicted_labels = model.get_predicted_token_labels(
    predictions=preds,
    label_map=label_map,
    dataset=sample_dataset
)

for i in range(len(sample_text)):
    print("\n", sample_text[i])
    print(pd.DataFrame({"tokens": sample_tokens[i] , "labels":predicted_labels[i]}))  

NameError: name 'processor' is not defined

## For Testing

In [ ]:
report_splits = report.split('\n')[-2].split()

sb.glue("precision", float(report_splits[2]))
sb.glue("recall", float(report_splits[3]))
sb.glue("f1", float(report_splits[4]))